In [ ]:
#IMPORT FUNCTIONS
from m6predictFunctions import *

In [ ]:
#Set Variables
GPU=2
h5Flank=10000
modelFlank=80
numOut_Classes=3 
batch_size_perGPU=18

In [ ]:
h5halfFlank=int(h5Flank/2)
halfFlankL=int(modelFlank/2)

assert h5halfFlank-halfFlankL>=0

#adjust model based on size of halfFlankL
inputs = Input(shape=(None,4))
x = Conv1D(32,(1,))(inputs)
skip = Conv1D(32,(1,))(x)
x=resnet(x,32,11,1,4)
skip=add([Conv1D(32,(1,))(x),skip])
if modelFlank >= 400:
    x=resnet(x,32,11,4,4)
    skip=add([Conv1D(32,(1,))(x),skip])
if modelFlank >= 2000:
    x=resnet(x,32,21,10,4)
    skip=add([Conv1D(32,(1,))(x),skip])
if modelFlank >= 10000:
    x=resnet(x,32,41,25,4)
    skip=add([Conv1D(32,(1,))(x),skip])
skip=Cropping1D(halfFlankL)(skip)
predictions=Conv1D(numOut_Classes,(1,),activation='softmax')(skip)

model = Model(inputs,predictions)
model = make_parallel(model,GPU)

if numOut_Classes < 3:
    model.compile(optimizer='adam',loss=categorical_crossentropy_2d_2ctgs)
else: 
    model.compile(optimizer='adam',loss=categorical_crossentropy_2d_3ctgs)

#read h5 files

InpF=h5py.File('Inp_train.h5', 'r')
TgtF=h5py.File('Tgt_train.h5', 'r')

num_idx = len(InpF.keys())
idx_all = np.random.permutation(num_idx)
idx_train = idx_all[:int(0.9*num_idx)]
idx_valid = idx_all[int(0.9*num_idx):]

EPOCH_NUM = 10*len(idx_train)
val_loss=[]
earlyStopPatience=1

for epoch_num in range(EPOCH_NUM):
    idx = np.random.choice(idx_train)
    Inp=InpF[str(idx)][:]
    Tgt=TgtF[str(idx)][:]

    rem = Inp.shape[0] % GPU
    if rem != 0:
        Inp=Inp[:-rem]
        Tgt=Tgt[:-rem]
    
    if h5halfFlank-halfFlankL > 0:
        Inp=Inp[:,h5halfFlank-halfFlankL:-h5halfFlank+halfFlankL]
    
    model.fit(Inp,Tgt,batch_size=batch_size_perGPU*GPU,verbose=1)
    
    if (epoch_num+1) % len(idx_train) == 0:
        #calculate validation data
        Tk1s,Tk2s,PR1s,PR2s = [],[],[],[]
        for idxV in idx_valid:
            Iv=InpF[str(idxV)][:]
            Tv=InpF[str(idxV)][:]
            Tk1,Tk2,PR1,PR2 = topK_auPRC(model,Inp,Tgt,numOut_Classes)
            Tk1s.append(Tk1),Tk2s.append(Tk2),PR1s.append(PR1),PR2s.append(PR2)
        print 'Top_k m6A:{}  Top_k 3rd class:{}  PR m6A:{}  PR 3rd class:{}'.format(
            np.mean(Tk1s),np.mean(Tk2s),np.mean(PR1s),np.mean(PR2s))
        val_loss.append(np.mean(Tk1s+Tk2s+PR1s+PR2s))
        #earlyStop
        val_maxI=np.argmax(val_loss)
        if len(val_loss)-1-val_maxI > earlyStopPatience:
            print 'val_maxI is {}'.format(val_maxI)
            break 
        #save model 
        model.save('model_weights_{}.h5'.format((epoch_num+1)/len(idx_train)-1))
        #change learning rate 
        if (epoch_num+1) >= 6*len(idx_train):
            kb.set_value(model.optimizer.lr,
                         0.5*kb.get_value(model.optimizer.lr))

os.system('cp model_weights_{}.h5 model_weights_BEST.h5'.format(val_maxI))